In [297]:
#pip install msedge-selenium-tools selenium==3.141

In [298]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
from time import sleep
from datetime import datetime
from openpyxl import Workbook
import csv

In [299]:
# star wars: squadrons (269790)
# scrap mechanic (387990)
game_id = 656970

In [300]:
template = 'https://steamcommunity.com/app/{}/reviews?p=1&browsefilter=toprated'
template_with_language = 'https://steamcommunity.com/app/{}/reviews?p=1&browsefilter=toprated&filterLanguage=english'

url = template.format(game_id)

In [301]:
# setup driver
options = Options()
options.use_chromium = True
driver = webdriver.Chrome(options=options)

In [302]:
driver.maximize_window()
driver.get(url)

In [303]:
# get current position of y scrollbar
last_position = driver.execute_script("return window.pageYOffset;")

reviews = []
review_ids = set()
running = True

while running:
    # get cards on the page
    cards = driver.find_elements_by_class_name('apphub_Card')

    for card in cards[-20:]:  # only the tail end are new cards

        # gamer profile url
        profile_url = card.find_element_by_xpath('.//div[@class="apphub_friend_block"]/div/a[2]').get_attribute('href')

        # steam id
        steam_id = profile_url.split('/')[-2]
        
        # check to see if I've already collected this review
        if steam_id in review_ids:
            continue
        else:
            review_ids.add(steam_id)

        # username
        user_name = card.find_element_by_xpath('.//div[@class="apphub_friend_block"]/div/a[2]').text

        # language of the review
        date_posted = card.find_element_by_xpath('.//div[@class="apphub_CardTextContent"]/div').text
        review_content = card.find_element_by_xpath('.//div[@class="apphub_CardTextContent"]').text.replace(date_posted,'').strip()    

        # review length
        review_length = len(review_content.replace(' ', ''))    

        # recommendation
        thumb_text = card.find_element_by_xpath('.//div[@class="reviewInfo"]/div[2]').text
        thumb_text    

        # amount of play hours
        play_hours = card.find_element_by_xpath('.//div[@class="reviewInfo"]/div[3]').text
        play_hours    

        # save review
        review = (steam_id, profile_url, review_content, thumb_text, review_length, play_hours, date_posted)
        reviews.append(review)    
        
    # attempt to scroll down thrice.. then break
    scroll_attempt = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
        sleep(0.5)
        curr_position = driver.execute_script("return window.pageYOffset;")
        
        if curr_position == last_position:
            scroll_attempt += 1
            sleep(0.5)
            
            if scroll_attempt >= 30:
                running = False
                break
        else:
            last_position = curr_position
            break  # continue scraping the results

# shutdown the web driver
driver.close()

In [304]:
# save the file to Excel Worksheet
wb = Workbook()
ws = wb.worksheets[0]
ws.append(['SteamId', 'ProfileURL', 'ReviewText', 'Review', 'ReviewLength(Chars)', 'PlayHours', 'DatePosted'])
for row in reviews:
    ws.append(row)
    
today = datetime.today().strftime('%Y%m%d')    
wb.save(f'Steam_Reviews_{game_id}_{today}.xlsx')    
wb.close()

In [305]:
# save the file to a CSV file
today = datetime.today().strftime('%Y%m%d')   
with open(f'Steam_Reviews_{game_id}_{today}.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['SteamId', 'ProfileURL', 'ReviewText', 'Review', 'ReviewLength(Chars)', 'PlayHours', 'DatePosted'])
    writer.writerows(reviews)